In [1]:
import os, json, copy
import pandas as pd
from tqdm import tqdm

In [2]:
path_2 = '주제별 텍스트 일상 대화 데이터/'
file_list_2 = os.listdir(path_2)


li = []
for i in tqdm(range(len(file_list_2))):
    folder = file_list_2[i]

    path_1 = '주제별 텍스트 일상 대화 데이터/'+folder+'/'
    file_list_1 = os.listdir(path_1)

    for n in range(len(file_list_1)):
        with open(path_1+file_list_1[n], 'r', encoding='utf-8') as file:
            temp_data = json.load(file)
            li.append(temp_data)

100%|██████████| 6/6 [00:07<00:00,  1.17s/it]


In [3]:
def preprocessing_list(x):
    dial_list = x['info'][0]['annotations']['text'].split('\n')
    temp_list = []; cnt = 0
    temp_list = copy.deepcopy(dial_list)
    for i in range(len(dial_list)):
        i -= cnt
        if temp_list[i][0] not in ['1','2','3','4','5'] :
            temp_list[i-1] += temp_list[i].lstrip()
            temp_list.pop(i); cnt += 1
    return temp_list

In [4]:
dialog = pd.DataFrame(columns=['Q','Q_intent', 'A', 'A_intent'])

for i in tqdm(range(len(li))):
    sen = preprocessing_list(li[i])
    intent = li[i]['info'][0]['annotations']['lines']
    for j in range(len(sen)):
        if len(sen) != len(intent):
            print(f"len(sen) != len(intent) : {li[i]['dataset']['name']}")
            break
        else :
            for k in range(1,6):
                if j+k >= len(sen)-1:
                    break
                elif sen[j][0] == sen[j+k][0]:
                    break

                if intent[j]['speechAct'] != intent[j+k]['speechAct']:
                    # dialog = pd.concat([dialog,pd.DataFrame({'Q' : [sen[j].split(':')[-1].lstrip()], 'Q_intent' : intent[j]['speechAct'], 'A' : [sen[j+k].split(':')[-1].lstrip()], 'A_intent' : intent[j+k]['speechAct']})],ignore_index=True)
                    try:
                        dialog = pd.concat([dialog,pd.DataFrame({'Q' : [sen[j].split(':')[-1].lstrip()],
                                                                 'Q_intent' : intent[j]['speechAct'],
                                                                 'A' : [sen[j+k].split(':')[-1].lstrip()],
                                                                 'A_intent' : intent[j+k]['speechAct']})], ignore_index=True)
                    except :
                        print(li[i]['dataset']['name'])
                        print(f'대화번호i:{i} / 문장번호j:{j} / 쌍번호k:{k}')
dialog

 13%|█▎        | 3508/26215 [00:51<07:02, 53.72it/s] 

KAKAO_1174_20_set
대화번호i:3499 / 문장번호j:22 / 쌍번호k:1


 15%|█▍        | 3876/26215 [01:06<29:50, 12.48it/s]

In [140]:
dialog.to_csv('dataset/dialog_chatbot_source.csv')